In [3]:
from AGCRLEnv import AGCRLEnv
from DQNAgent import DQNAgent
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
from tqdm import tqdm
import time
import tensorflow as tf

In [4]:

with open('observations.pickle', 'rb') as handle:
    obs = pickle.load(handle)
with open('actions.pickle', 'rb') as handle:
    actions = pickle.load(handle)

In [5]:
assim_rl_actionspace=np.linspace(0,100,21)
env=AGCRLEnv(obs,actions,"assim_sp",assim_rl_actionspace)

In [6]:
env.resetinit()

%time                       43815.003906
AssimLight                      0.000000
BlackScr                       85.000000
CO2air                        470.000000
Cum_irr                         0.300000
EC_drain_PC                     1.900000
EnScr                          96.000000
HumDef                          7.000000
PipeGrow                       55.900002
PipeLow                        57.500000
Rhair                          60.799999
Tair                           20.600000
Tot_PAR                         0.000000
Tot_PAR_Lamps                   0.000000
VentLee                         1.000000
Ventwind                        0.000000
co2_dos                         0.000500
pH_drain_PC                     7.300000
water_sup                       4.000000
ProdA                           0.050000
ProdB                           0.100000
avg_nr_harvested_trusses        0.100000
Truss development time         50.000000
Nr_fruits_ClassA                0.000000
Weight_fruits_Cl

In [7]:


# Environment settings
EPISODES = 1
# Exploration settings
epsilon = 1  # not a constant, going to be decayed
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001

#  Stats settings
AGGREGATE_STATS_EVERY = 500  # steps
SHOW_PREVIEW = False
SAVE_MODEL_EVERY=5000
MODEL_NAME="AGCRL_ASSIM_ACCREWARD"
MIN_REWARD = -5000

In [8]:
# Create models folder
if not os.path.isdir('models'):
    os.makedirs('models')

In [9]:
env.observation_space

(39,)

In [10]:
# new_model = tf.keras.models.load_model('models/AGCRL_ASSIM___100.00max___60.34avg_-100.00min__1618374625.model')

In [11]:
agent = DQNAgent(env,env.action_space)
# agent = DQNAgent(env,env.action_space)

In [12]:
env.resetinit(),env.teamindex

(%time                       43815.003906
 AssimLight                      0.000000
 BlackScr                       85.000000
 CO2air                        470.000000
 Cum_irr                         0.300000
 EC_drain_PC                     1.900000
 EnScr                          96.000000
 HumDef                          7.000000
 PipeGrow                       55.900002
 PipeLow                        57.500000
 Rhair                          60.799999
 Tair                           20.600000
 Tot_PAR                         0.000000
 Tot_PAR_Lamps                   0.000000
 VentLee                         1.000000
 Ventwind                        0.000000
 co2_dos                         0.000500
 pH_drain_PC                     7.300000
 water_sup                       4.000000
 ProdA                           0.050000
 ProdB                           0.100000
 avg_nr_harvested_trusses        0.100000
 Truss development time         50.000000
 Nr_fruits_ClassA                0

In [13]:
ep_rewards=[]
acc_avg_reward=[0]
for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):
    step=0
    # Update tensorboard step every episode
    agent.tensorboard.step = episode

    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1

    # Reset environment and get initial state
    current_state = env.reset()

    # Reset flag and start iterating until episode ends
    done = False
    while not done:
        step+=1
        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
        else:
            # Get random action
            action = np.random.randint(0, len(env.action_space))

        new_state, reward, done = env.step(action)

        # Every step we update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)

        current_state = new_state
        ep_rewards.append(reward)
#         print(step%AGGREGATE_STATS_EVERY)
        if (step % AGGREGATE_STATS_EVERY)==0:
            print(step)
            
            average_reward = sum(ep_rewards)/len(ep_rewards)
            min_reward = min(ep_rewards)
            max_reward = max(ep_rewards)
            ar=average_reward
            arval=acc_avg_reward[-1]+ar
            acc_avg_reward.append(arval)
            agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon,acc_avg_reward=arval)
            ep_rewards=[]
            # Save model, but only when min reward is greater or equal a set value
            if min_reward >= MIN_REWARD and step%SAVE_MODEL_EVERY==0:
                agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')

        if epsilon > MIN_EPSILON:
            epsilon *= EPSILON_DECAY
            epsilon = max(MIN_EPSILON, epsilon)

  0%|          | 0/1 [00:00<?, ?episodes/s]

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/AGCRL_ASSIM_ACCREWARD___100.00max__-22.10avg_-100.00min__1618420026.model/assets
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
INFO:tensorflow:Assets written to: models/AGCRL_ASSIM_ACCREWARD___100.00max__-33.13avg_-100.00min__1618420396.model/assets
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
INFO:tensorflow:Assets written to: models/AGCRL_ASSIM_ACCREWARD___100.00max___-9.81avg_-100.00min__1618420762.model/assets
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
INFO:tensorflow:Assets written to: models/AGCRL_ASSIM_ACCREWARD___100.00max___65.16avg_-100.00min__1618421101.model/assets
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
INFO:tensorflow:Assets written to: models/AGCRL_ASSIM_ACCREWARD___100.00max___45.70avg_-100.00min__1618421449.model/assets
25500
26000
26500
27000


100%|##########| 1/1 [58:36<00:00, 3516.63s/episodes]


In [35]:
sum(ep_rewards)/len(ep_rewards)

-44.65564738292011